<a href="https://colab.research.google.com/github/DesmondChoy/llm_tutorials/blob/main/Llama3_Researcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taken from [Matt Shumer](https://github.com/mshumer/ai-researcher/blob/main/Claude_Researcher.ipynb)'s github.

https://console.groq.com/docs/text-chat

Can we use Llama3-70B to generate the subtopics, and have Claude 3 Opus as the boss and summarize?

In [1]:
!pip install serpapi groq -qq

import requests
import json
import time
import ast
import re
from groq import Groq
from google.colab import userdata


ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')  # Replace with your Anthropic API key
SERP_API_KEY = userdata.get('SERP_API_KEY')  # Replace with your SERP API key
client = Groq(api_key = userdata.get('GROQ_API_KEY'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


Let's put together a notebook to flesh out the workflow, before refactoring it.  




# Llama3 Chat Template

[Claude model names](https://docs.anthropic.com/claude/docs/models-overview#:~:text=request%20using%20Claude.-,Claude%203%3A%20A%20new%20generation%20of%20AI,-Model) and [models supported by Groq](https://console.groq.com/docs/models).  

Models consideration: Long context windows are preferable.

JSON mode is a beta feature that guarantees all chat completions are valid JSON.  
Usage: Set "response_format": {"type": "json_object"} in your chat completion request

In [8]:
def generate_text(
    research_topic, model="llama3-70b-8192", max_tokens=2000, temperature=0.7
):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a world-class researcher. Analyze the user prompt and generate a well-structured report.",
            },
            {"role": "user", "content": research_topic},
        ],
        model=model,
        # response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=0.95,
        stop=None,
        stream=False,
    )

    return chat_completion.choices[0].message.content

# print(chat_completion.choices[0].message.content)

# Define function to call search engine and search

The `search_web()` function is defined to perform a web search using the SERP API. It takes a search term as a parameter, constructs the API URL, sends a GET request to the SERP API, and returns the search results as JSON data.

In [53]:
def search_web(search_term):
    url = f"https://serpapi.com/search.json?q={search_term}&api_key={SERP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(data)
    return data

# Generate sub topics

The `generate_subtopic_report()` function is the core of the research automation process. It takes a subtopic as input and generates a comprehensive report on that subtopic. Here's a breakdown of its steps:
+ It generates initial search queries for the subtopic using the `generate_text()` function.
+ It performs multiple rounds of web searches using the `search_web()` function and the generated queries.
+ After each search round, it generates additional search queries based on the search results and previous queries.
+ It generates an initial report using the accumulated search data and the `generate_text()` function.
+ It analyzes the initial report, identifies areas that need more detail, and generates additional search queries to fill in the gaps.
+ It updates the report by incorporating the new information from the additional searches.
+ It generates boss feedback on the report using the `generate_text()` function.
+ Based on the boss's feedback, it generates a final round of search queries to find missing information and address areas that need further elaboration.
+ It updates the report with the final information and returns the final report.

Specify subtopic (research topic?):

In [36]:
subtopic = "Considerations for setting up an aquarium for the first time."


Generate 3 search queries to gather information on the subtopic 'Considerations for setting up an aquarium for the first time.'. 
Return your queries in a Python-parseable list. Return only the list and nothing else.



In [50]:
search_data = []
all_queries = []

research_topic = f"""
Generate 3 search queries to gather information on the subtopic '{subtopic}'.
Return your queries in a Python-parseable list. Return only the list and nothing else.
"""

initial_queries = ast.literal_eval(generate_text(research_topic))
all_queries.extend(initial_queries)

In [54]:
for i in range(3):
    print(f"Performing search round {i+1} for subtopic: {subtopic}...")
    for query in initial_queries:
        search_results = search_web(query)
        search_data.append(search_results)

    print(f"Generating additional search queries for subtopic: {subtopic}...")
    additional_queries_i_prompt = f"""
        Here are the search results so far for the subtopic '{subtopic}':\n\n{str(search_data)}\n\n---\n\n
        Here are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\n
        Based on the search results and previous queries, generate 3 new and unique search queries to expand the knowledge on the subtopic '{subtopic}'.
        Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\"
        """
    additional_queries_i = ast.literal_eval(
        "[" + generate_text(additional_queries_i_prompt).split("[")[1]
    )
    all_queries.extend(additional_queries_i)


Performing search round 1 for subtopic: Considerations for setting up an aquarium for the first time....
{'search_metadata': {'id': '662bcda2a3f4ef2496dfc2a9', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/c09840a88584d40c/662bcda2a3f4ef2496dfc2a9.json', 'created_at': '2024-04-26 15:52:02 UTC', 'processed_at': '2024-04-26 15:52:02 UTC', 'google_url': 'https://www.google.com/search?q=setting+up+a+first+aquarium+checklist&oq=setting+up+a+first+aquarium+checklist&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/c09840a88584d40c/662bcda2a3f4ef2496dfc2a9.html', 'total_time_taken': 2.92}, 'search_parameters': {'engine': 'google', 'q': 'setting up a first aquarium checklist', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'setting up a first aquarium checklist', 'total_results': 921000, 'time_taken_displayed': 0.35, 'organic_results_state': 'Results for exact spelling'}, 'inline_videos': [{'position

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hw818v11f27rardkwkfenrgb` on tokens per minute (TPM): Limit 7000, Used 0, Requested ~29953. Please try again in 3m16.739999999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
#@title og generate_subtopic_report

def generate_subtopic_report(subtopic):
    search_data = []
    all_queries = []

    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"""
    Generate 3 search queries to gather information on the subtopic '{subtopic}'.
    Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\"
    """
    initial_queries = ast.literal_eval('[' + generate_text(initial_queries_prompt).split('[')[1])
    print(initial_queries)
    all_queries.extend(initial_queries)

    for i in range(3):
        print(f"Performing search round {i+1} for subtopic: {subtopic}...")
        for query in initial_queries:
            search_results = search_web(query)
            search_data.append(search_results)

        print(f"Generating additional search queries for subtopic: {subtopic}...")
        additional_queries_i_prompt = f"""
        Here are the search results so far for the subtopic '{subtopic}':\n\n{str(search_data)}\n\n---\n\n
        Here are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\n
        Based on the search results and previous queries, generate 3 new and unique search queries to expand the knowledge on the subtopic '{subtopic}'.
        Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\"
        """
        additional_queries_i = ast.literal_eval('[' + generate_text(additional_queries_i_prompt).split('[')[1])
        all_queries.extend(additional_queries_i)
        #initial_queries = additional_queries
        #all_queries = initial_queries

    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"""
    When writing your report, make it incredibly detailed, thorough, specific, and well-structured.
    Use Markdown for formatting. Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}':\n\n{str(search_data)}
    """
    report = generate_text(report_prompt, max_tokens=4000)

    for i in range(3):
        print(f"Analyzing report and generating additional searches (Round {i+1}) for subtopic: {subtopic}...")
        analysis_prompt = f"""
        Analyze the following report on the subtopic '{subtopic}' and identify areas that need more detail or further information:\n\n{report}\n\n---\n\n
        Here are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\n
        Generate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas.
        Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\"
        """
        additional_queries_ii = ast.literal_eval('[' + generate_text(analysis_prompt).split('[')[1])
        all_queries.extend(additional_queries_ii)

        round_search_data = []
        for query in additional_queries_ii:
            search_results = search_web(query)
            round_search_data.append(search_results)

        print(f"Updating report with additional information (Round {i+1}) for subtopic: {subtopic}...")
        update_prompt = f"""
        Update the following report on the subtopic '{subtopic}' by incorporating the new information from the additional searches.
        Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data for this round:\n\n{str(round_search_data)}\n\n---\n\n
        Generate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting.
        """
        report = generate_text(update_prompt, max_tokens=4000)

    print(f"Generating boss feedback for subtopic: {subtopic}...")
    feedback_prompt = f"""
    Imagine you are the boss reviewing the following report on the subtopic '{subtopic}':\n\n{report}\n\n---\n\n
    Provide constructive feedback on what information is missing or needs further elaboration in the report.
    Be specific and detailed in your feedback.
    """
    feedback = generate_text(feedback_prompt, max_tokens=1000)

    print(f"Generating final round of searches based on feedback for subtopic: {subtopic}...")
    final_queries_prompt = f"""
    Based on the following feedback from the boss regarding the subtopic '{subtopic}':\n\n{feedback}\n\n---\n\n
    Generate 3 search queries to find the missing information and address the areas that need further elaboration.
    Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\"
    """
    final_queries = ast.literal_eval('[' + generate_text(final_queries_prompt).split('[')[1])
    all_queries.extend(final_queries)

    final_search_data = []
    for query in final_queries:
        search_results = search_web(query)
        final_search_data.append(search_results)

    print(f"Updating report with final information for subtopic: {subtopic}...")
    final_update_prompt = f"""
    Update the following report on the subtopic '{subtopic}' by incorporating the new information from the final round of searches
    based on the boss's feedback:\n\n{report}\n\n---\n\nFinal search data:\n\n{str(final_search_data)}\n\n---\n\n
    Generate the final report that addresses the boss's feedback and includes the missing information. Use Markdown for formatting.
    """
    final_report = generate_text(final_update_prompt, max_tokens=4000)

    print(f"Final report generated for subtopic: {subtopic}!")
    return final_report

## Reconstructing into shorter functions

The `generate_comprehensive_report()` function takes the research topic and the subtopic reports as input. It generates a comprehensive report by combining the subtopic reports, ensuring that all important aspects are covered and the report is well-structured and coherent.

In [ ]:
def generate_comprehensive_report(topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Generate a comprehensive report on the topic '{topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report = generate_text(comprehensive_report_prompt, model="claude-3-opus-20240229", max_tokens=4000)

    print("Comprehensive report generated!")
    return comprehensive_report



The script prompts the user to enter the research topic.  

It generates a checklist of subtopics for the given research topic using the `generate_text()` function.  

For each subtopic in the checklist, it calls the `generate_subtopic_report()` function to generate a report.  

After generating reports for all subtopics, it calls the `generate_comprehensive_report()` function to combine the subtopic reports into a comprehensive report.  

Finally, it saves the comprehensive report to a file named "comprehensive_report.txt".

In [ ]:
# User input
research_topic = input("Enter the research topic: ")

# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")

# Generate reports for each subtopic
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(subtopic)
    subtopic_reports.append(subtopic_report)

# Combine subtopic reports into a comprehensive report
comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))

# Save the comprehensive report to a file
with open("comprehensive_report.txt", "w") as file:
    file.write(comprehensive_report)

print("Comprehensive report saved as 'comprehensive_report.txt'.")